In [15]:
import numpy as np
from numba import njit, prange
from scipy import sparse

In [19]:
def laplacian(N, dx):
    _A_1 = sparse.eye(N)
    D_2 = sparse.eye(N, k=1) + sparse.eye(N, k=-1) - 2 * _A_1
    L = 1 / (dx**2) * (sparse.kron(D_2, _A_1) + sparse.kron(_A_1, D_2))
    return L

In [20]:
laplacian(10, 0.1)

<100x100 sparse matrix of type '<class 'numpy.float64'>'
	with 460 stored elements in Compressed Sparse Row format>

In [18]:
10 * 10

100

In [29]:
A = np.random.rand(3, 3)

In [34]:
np.diag(A)

array([0.45409301, 0.25852559, 0.20651691])

In [31]:
A

array([[0.45409301, 0.21846916, 0.20702795],
       [0.03598636, 0.25852559, 0.83146031],
       [0.65242193, 0.82114557, 0.20651691]])